## MakerPi

This is a board with an RP 2040 at the core.  A great intro to the board and micropython is [here](https://www.coderdojotc.org/micropython/intro/01-about/) and even more info (how to connect etc) [here](https://dmccreary.medium.com/the-cytron-maker-pi-rp2040-robotics-board-b1dc7f0eab34).  Unfortunately you will not be able to program this board directly from the ROS server until later.  So for now, copy/paste your code into [here](https://pyrepl.web.app/).  (This UI was developed by Gabe last summer at the CEEO).  You might need to download firmware - follow these teps if you cannot see your board in Chrome.  

1. turn the board off, old down the BOOT button and then turn the board on (connect to the computer) and let go
2. look to see that the board showed up as a USB key on your computer
3. download the firmware from [here](https://micropython.org/download/rp2-pico/)
4. drag the firmware over to the USB Key, it will copy it over and reboot the board
5. go to the software - all should work

First check to make sure the REPL is talking...

In [ ]:
2+2

## Turning on an LED

To turn on a light, we need to tell the processor which pin to toggle - we can set up the pins from a library called machine.  Set it up to be an output (send voltage out rather than receive it)

In [ ]:
import machine
import time

fred = machine.Pin(0, machine.Pin.OUT)
fred.on()
time.sleep(1)
fred.off()

Can you edit the code above to run through the first 8 lights?  A python for loop looks like:

In [ ]:
for i in range(8):
    print(i)

Or, if you want to play with arrays, you can do it like this... pins on microprocessors are usually called GPIO pins (general purpose input/output pins).

In [ ]:
import machine
import time

# 13 blue LEDs
led_gpio = [0,1,2,3,4,5,6,7,16,17,26,27,28]
ports = []

# Make the ports array
for led in led_gpio:
    ports.append(machine.Pin(led, machine.Pin.OUT))

for i in range(len(ports)):
    led_ports[i].high()
    time.sleep(0.1)
    led_ports[i].low()

And then there are nightrider lights - can you figure this code out?  Try changing the speed and order of the lights.

In [ ]:
import machine
import time

# 13 blue LEDs
led_gpio = [0,1,2,3,4,5,6,7,16,17,26,27,28]
ports = []

# Make the ports array
for led in led_gpio:
    ports.append(machine.Pin(led, machine.Pin.OUT))

wait=0.2

while True:
    # move down
    for i in range(len(ports)):
        ports[i].high()
        time.sleep(wait)
        ports[i].low()
    # move up
    for i in range(len(ports) - 1, 0, -1):
        ports[i].high()
        time.sleep(wait)
        ports[i].low()

## Reading the light sensor

First, find the pin you plugged the light sensor into and then setup the pins.  You will use an *analog to digital converter* to convert the voltage into a number.  Use the link at the top of the page to figure out which pin is which (or look at the board).  Notice it is reading the voltage as an unsigned 16 bit integer - or values from 0 to 65535 (which is $2^{16}$)

In [ ]:
import machine
import time

photo_pin = machine.ADC(28)  # port 7

while not done:
    val = photo_pin.read_u16()
    print(val)
    time.sleep(.2)

## Controlling a servo motor

Now try to make a servo motor rock back and forth.  Notice that you have 65536 possible values ($2^{16}$) spread over a 1 msec duty cycle (from 1 to 2 msec with 1.5 msec being the middle).  This will vary a little from servo to servo - so you might have to calibrate.  Then you set it up to send *pwm* out the port at 50 Hz (20 msec) and just loop through rotating in 1 degree increments.

In [ ]:
import machine
import time

#-------initialize things
period = 20 #msec
frequency = int(1000/period)
min = int(65536/period * 1.0)
mid = int(1.5 * period)
max = int(2.0 * period)

pwm = machine.PWM(machine.Pin(15))
pwm.freq(frequency)

#---------define function
def servo(angle=0):
    dutycycle = int(((max - min)/180)*angle)+min
    pwm.duty_u16(dutycycle)
    
#----------main code
servo(-90)

while True:
    for angle in range (-90,90,1):
        servo(angle)
        time.sleep(0.01)
    for angle in range (90,-90,-1):
        servo(angle)
        time.sleep(0.01)


## Adding sound

And if you want to add a buzzer, it is on pin 22 (make sure the switch is switched to on next to the buzzer).  Play around with different duty cycles - can you explain the result?

In [ ]:
buzzer = machine.PWM(machine.Pin(22))
buzzer.freq(440)
buzzer.duty_u16(int(65536/2))

or make it a function that you can call

In [ ]:
def beep(frequency = 440):
    buzzer = machine.PWM(machine.Pin(22))
    buzzer.freq(frequency)
    buzzer.duty_u16(int(65536/2))
    time.sleep(0.5)
    buzzer.duty_u16(0)

and try calling it

In [ ]:
beep(880)

## Interrupts (callbacks)

Next - lets set up an interrupt (or callback) to stop the loop when the button is pushed.  You could do this also by just always checking the state of the button in the while loop - but then you have to have the button held down when the check happens, this will interrupt the code no matter where it is.

In [ ]:
from machine import Pin
import machine, micropython
import time

#-------initialize things
micropython.alloc_emergency_exception_buf(100) # allows error reporting for callbacks

light = [Pin(0, Pin.OUT),Pin(1, Pin.OUT),Pin(2, Pin.OUT)]
button1 = machine.Pin(20, machine.Pin.IN, machine.Pin.PULL_DOWN)
done = False
delay = 1

def button1_pressed(change):
    global done, delay
    print('pushed')
    time.sleep(0.1)
    done = True  #stop the loop
    delay = 0 #hurry up and finish the loop
    button1.irq(None) #end the interrupt

#---------go
button1.irq(handler=button1_pressed, trigger=machine.Pin.IRQ_FALLING)  #start interrupt

while not done:
    time.sleep(0.01)
    for led in light:
        led.on()
    time.sleep(delay)
    for led in light:
        led.off()
    time.sleep(delay)

## Putting it all together

Try to rewrite your code that would kick the soccer ball - only this time beep when you see a light change as well as kick